# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [21]:
# Get the datasets
!curl.exe --output train.dat http://huang.eng.unt.edu/CSCE-5218/train.dat
!curl.exe --output test.dat http://huang.eng.unt.edu/CSCE-5218/test.dat


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 11244  100 11244    0     0  92294      0 --:--:-- --:--:-- --:--:-- 92925
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2844  100  2844    0     0  48601      0 --:--:-- --:--:-- --:--:-- 49034


In [1]:
# Take a peek at the datasets
!more train.dat
!more test.dat

A1      A2      A3      A4      A5      A6      A7      A8      A9      A10     A11     A12     A13     
1       1       0       0       0       0       0       0       1       1       0       0       1       0
0       0       1       1       0       1       1       0       0       0       0       0       1       0
0       1       0       1       1       0       1       0       1       1       1       0       1       1
0       0       1       0       0       1       0       1       0       1       1       1       1       0
0       1       0       0       0       0       0       1       1       1       1       1       1       0
0       1       1       1       0       0       0       1       0       1       1       0       1       1
0       1       1       0       0       0       1       0       0       0       0       0       1       0
0       0       0       1       1       0       1       1       1       0       0       0       1       0
0       0       0       0       0       0      

### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [54]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    instances = []
    with open(file_name, "r") as f:
        for instance in f.readlines():
            if not re.search('\t', instance): continue
            try:
                instance = list(map(int, instance.strip().split('\t')))
            except ValueError:
                print(f"Skipping instance {instance} due to invalid input value.")
                continue
            # Add a dummy input so that w0 becomes the bias
            instance = [-1] + instance 
            instances.append(instance)
    return instances


def dot_product(array1, array2):
    #TODO: Return dot product of array 1 and array 2
    return sum(x * y for x, y in zip(array1, array2))
# this returns the dot product of array 1 and array 2

def sigmoid(x):
    #TODO: Return outpout of sigmoid function on x
    return 1 / (1 + math.exp(-x))
#returns the output of sigmoid function

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    #TODO: return the output of the model 
    dot_prod = dot_product(weights, instance)
    return sigmoid(dot_prod)

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    #TODO: return the prediction of the model
    output_value = output(weights, instance)
    
    # Compare the output to the threshold
    if output_value >= 0.5:
        return 1
    else:
        return 0


# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):

    # Initialize weights and iterate for a fixed number of epochs
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            # Forward propagation step
            in_value = dot_product(weights, instance)
            output = sigmoid(in_value)
            error = instance[-1] - output
            # Backward propagation step
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

## Run it

In [55]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

Skipping instance A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
 due to invalid input value.
Skipping instance A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
 due to invalid input value.
#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)
We want to utilize the output as the input to the weight update algorithm in the next phase, therefore we compute the output using the sigmoid function rather than the predict function. The sigmoid function produces a continuous output between 0 and 1, while the predict function produces a binary result of either 0 or 1. We get a more detailed representation of the result when we use the sigmoid function, which enables us to accurately modify the weights. For the purpose of computing the gradient of the error with regard to the weights, the error term error = instance[-1] - output specifically refers to the difference between the actual target output and the anticipated output from the sigmoid function.



### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 
```import random

tr_percent = [5, 10, 25, 50, 75, 100]
num_epochs = [5, 10, 20, 50, 100]
lr = [0.005, 0.01, 0.05]

instances_te = read_data("test.dat")

for tp in tr_percent:
    num_instances = int(len(instances_tr) * tp / 100)
    for e in num_epochs:
        for l in lr:
            instances_sub = random.sample(instances_tr, num_instances)
            weights = train_perceptron(instances_sub, l, e)
            accuracy = get_accuracy(weights, instances_te)
            print(f"# tr: {num_instances:3}, epochs: {e:3}, learning rate: {l:.3f}; "
                  f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")
```

OUTPUT
```
# tr:   9, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:   9, epochs:   5, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
# tr:   9, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
# tr:   9, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:   9, epochs:  10, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
# tr:   9, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
# tr:   9, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:   9, epochs:  20, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
# tr:   9, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
```


In [56]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

for lr in lr_array:
  for tr_size in tr_percent:
    for epochs in num_epochs:
      size =  round(len(instances_tr)*tr_size/100)
      pre_instances = instances_tr[0:size]
      weights = train_perceptron(pre_instances, lr, epochs)
      accuracy = get_accuracy(weights, instances_te)
    print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
            f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

Skipping instance A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
 due to invalid input value.
Skipping instance A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
 due to invalid input value.
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 200, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 74.0
#tr: 300, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 78.0
#tr: 400, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 77.0
#tr: 20, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 78.0
#tr

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

### TODO: Add your answer here (code and text)
A. To get the best accuracy with the test dataset, you do not necessarily need to train using the whole training dataset.The accuracy of the model will generally increase with the amount of training data you have, but there are some situations when adding more data may not enhance performance, particularly if the data is noisy or duplicated. The quantity of data and the amount of time and money needed to train the model are frequently trade-offs.In some cases, employing a smaller subset of the training data might aid in lowering overfitting and enhancing the model's capacity for generalization.

B. Although utilizing more training data, the second run's accuracy was worse than the first run's, which might be explained by the possibility that the additional data contains noise or irrelevant characteristics that impair the perceptron's performance. A straightforward linear classifier known as a perceptron is sensitive to unrelated and correlated data, which can result in overfitting and subpar generalization. Consequently, increasing the amount of data does not necessarily result in improved performance, particularly when the data is erratic or redundant. In order to optimize the perceptron's performance, it is frequently helpful to preprocess the data by deleting pointless features, balancing the classes, or normalizing the inputs.

C.By experimenting with various hyperparameters or employing more sophisticated models, it is feasible to get a greater accuracy. Because the dataset we are utilizing is hypothetical, we should be aware that it could not be an accurate representation of actual data. Also, we must take care to avoid overfitting the model to the training set of data because doing so might lead to worse performance on untried, fresh data. Thus, we should assess the final model on a test set that has not been used for training or validation and utilize a validation set to fine-tune the hyperparameters.

CODE

import random
import re
from math import exp

def read_data(file_name):
    with open(file_name, 'r') as f:
        instances = []
        for instance in f.readlines():
            if not re.search('\t', instance): continue
            instance = list(map(int, instance.strip().split('\t')))
            # Add a dummy input so that w0 becomes the bias
            instance = [-1] + instance 
            instances.append(instance)
        return instances

def dot_product(values, weights):
    return sum(value * weight for value, weight in zip(values, weights))

def sigmoid(in_value):
    return 1 / (1 + exp(-in_value))

def predict(weights, instance):
    in_value = dot_product(weights, instance)
    return sigmoid(in_value)

def train_perceptron(instances, lr, epochs):
    # Initialize weights to zero
    weights = [0] * len(instances[0])
    
    for epoch in range(epochs):
        for instance in instances:
            output = predict(weights, instance)
            error = instance[-1] - output
            weights = [weight + lr * error * value for value, weight in zip(instance, weights)]
    return weights

def test_perceptron(weights, instances):
    num_correct = 0
    for instance in instances:
        if predict(weights, instance) >= 0.5 and instance[-1] == 1:
            num_correct += 1
        elif predict(weights, instance) < 0.5 and instance[-1] == 0:
            num_correct += 1
    accuracy = num_correct / len(instances) * 100
    return accuracy

tr_percent = [5, 10, 25, 50, 75, 100]
num_epochs = [5, 10, 20, 50, 100]
lr = [0.005, 0.01, 0.05]

instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")

# Train and test with all hyperparameters
best_accuracy = 0
best_hyperparameters = None

for tr in tr_percent:
    num_instances = int(len(instances_tr) * tr / 100)
    for epoch in num_epochs:
        for learning_rate in lr:
            train_set = random.sample(instances_tr, num_instances)
            weights = train_perceptron(train_set, learning_rate, epoch)
            test_accuracy = test_perceptron(weights, instances_te)
            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy
                best_hyperparameters = (tr, epoch, learning_rate)

print(f"Best hyperparameters: tr = {best_hyperparameters[0]}, epochs = {best_hyperparameters[1]}, lr = {best_hyperparameters[2]}")
print(f"Best accuracy: {best_accuracy:.1f}")

D.No, it doesn't always make sense to train for additional epochs. While training for additional epochs might improve the model's performance, it also raises the possibility of overfitting, particularly when the dataset is limited. The model will cease improving and begin to overfit the training set of data after a predetermined number of epochs. To establish the ideal balance between training duration and model performance, it is frequently required to experiment with various numbers of epochs.